PART 1: Data Extraction

Pandas was used to extract the comma sperated value document into a pandas dataform. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#reading in the data
SCproperties = pd.read_csv('train.csv')
unique = pd.read_csv('unique_m.csv')

mats = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al',
       'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn',
       'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
       'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
       'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm',
       'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
       'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At',
       'Rn']

#combining all data into one dataframe
all_data= pd.concat([SCproperties, unique], axis=1, join_axes=[SCproperties.index])
#print(all_data.shape)
all_data = all_data.loc[:,~all_data.columns.duplicated()]
#print(all_data.shape)


PART 2: Visialization and Exploration of the Data

The CU-O SCs were given a ratio of 2+-0.333 to capture all HTCs with roughly a ratio of 2. 

The iron SCs were found by simply looking at the SCs that contained iron.

The conventional SCs were found by searching for all SCs with a critical temperature of less than 10 K. 

In [ ]:
clf = unique[(unique > 0)] #This gives a NAN for any value that is zero
y = clf.count() #These can then be counted with the count feature
#print(y)

#Removing the items that are out of place
y = y.drop("critical_temp", axis=0)
y = y.drop("material", axis=0)

#Putting into a dataframe for easiler sorting
y = pd.DataFrame(y)

#Sorting the dataframe from largest to smallest
y = y.sort_values(by = [0], ascending = False)

#Plotting the twenty most abundant materials, as plotting all of them clogs up the graph. 
plt.plot(y[0:20])
plt.title('Twenty most abundant materials in SCs')
plt.show()

print('Note: all of the elementents could be plotted, but the x-axis becomes unreadable as the number increases.')

In [ ]:
#Determining the High Temperature Conductors made of Copper and Oxygen
#with O-CU ratio roughly 2.0
HTC = all_data[(all_data.O/all_data.Cu < 2.333) & (all_data.O/all_data.Cu > 1.667)]
#HTC

#Finding superconductors with Iron in them
Fe = all_data[(all_data.Fe != 0)]
#Fe  

#Finding conventional superconductors
ConvSC = all_data[(all_data.critical_temp < 10)]
#ConvSC

#Labels for plots
item_y = 'mean_Density'
item_x = 'critical_temp'

types = [HTC,Fe,ConvSC]

#print('HTC')
x = HTC[item_x]
plt.xlabel(item_x)
plt.ylabel('Density')
hist = x.hist(density = True)
plt.title('High Temp SCs with Copper and Oxygen')
plt.show()

#print('Iron')
x = Fe[item_x]
plt.xlabel(item_x)
plt.ylabel('Density')
hist = x.hist(density = True)
plt.title('SCs containing Iron')
plt.show()

#print('Conventional Superconductor')
x = ConvSC[item_x]
plt.xlabel(item_x)
plt.ylabel('Density')
hist = x.hist(density = True)
plt.title('Conventional SCs with T_crit < 10 K')
plt.show()

#print('All Superconductors')
x = all_data[item_x]
plt.xlabel(item_x)
plt.ylabel('Density')
hist = x.hist(density = True)
plt.title('All SCs')
plt.show()

PART 3: Dimesionality Reduction

In the 'unique' dataset, there is one X value and one Y value that are the primary clusters.
In the 'properties' dataset, there is a wider spread, but they are mainly clustered at under 15000 in the X component. 

In [ ]:
import seaborn as sns; sns.set()
from sklearn.decomposition import PCA

#Project to two dimensions.
pcas = PCA(2)
#Preserve 50% of cumulative explained variance
pca = PCA(0.5)
SCproperties_fit = pca.fit(SCproperties)
prop_split = pcas.fit_transform(SCproperties)

#Plot
plt.scatter(prop_split[:, 0], prop_split[:, 1],
            c=SCproperties.critical_temp, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.xlabel('X Component')
plt.ylabel('Y Component')
plt.title('PCA')
plt.colorbar();


In [ ]:
#unique needs to have the "materials" column removed to be fitted. 
unique_pca = unique.drop(["material"], axis = 1)

#Project to two dimensions.
pcas = PCA(2)
#Preserve 50% of cumulative explained variance
pca = PCA(0.5)
unique_fit = pca.fit(unique_pca)
unique_split = pcas.fit_transform(unique_pca)

#Plot
plt.scatter(unique_split[:, 0], unique_split[:, 1],
            c=unique.critical_temp, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.xlabel('X Component')
plt.ylabel('Y Component')
plt.title('PCA')
plt.colorbar();


Part 4 - Random Forests


In [ ]:
from sklearn.model_selection import train_test_split

#Setting up the property tables for each category to be analyzed.

#All the materials
#The inclusion of critical temp and the matieral name threw errors, so they were removed. 
all_data_props = all_data.loc[:, all_data.columns != 'critical_temp']
all_data_props = all_data_props.loc[:, all_data_props.columns != 'material']

#High temp copper SCs
HTC_props = HTC.loc[:, HTC.columns != 'critical_temp']
HTC_props = HTC_props.loc[:, HTC_props.columns != 'material']

#Ferric SCs
Fe_props = Fe.loc[:, Fe.columns != 'critical_temp']
Fe_props = Fe_props.loc[:, Fe_props.columns != 'material']

#Conventional SCs
ConvSC_props = ConvSC.loc[:, ConvSC.columns != 'critical_temp']
ConvSC_props = ConvSC_props.loc[:, ConvSC_props.columns != 'material']

temp = all_data[['critical_temp']]
temp_HTC = HTC[['critical_temp']]
temp_Fe = Fe[['critical_temp']]
temp_ConvSC = ConvSC[['critical_temp']]

#Setting up the training and testing sets, using all of the data together, and then the individual categories data
data_train, data_test, target_train, target_test = train_test_split(all_data_props, temp, test_size = 0.3)

data_train_HTC, data_test_HTC, target_train_HTC, target_test_HTC = train_test_split(HTC_props, temp_HTC, test_size = 0.3, random_state = 100)

data_train_Fe, data_test_Fe, target_train_Fe, target_test_Fe = train_test_split(Fe_props, temp_Fe, test_size = 0.3, random_state = 100)

data_train_ConvSC, data_test_ConvSC, target_train_ConvSC, target_test_ConvSC = train_test_split(ConvSC_props, temp_ConvSC, test_size = 0.3, random_state = 100)


In [ ]:
#Import packages
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

#list(SCproperties)
params = list(SCproperties)

#Preprocessing - THe floats had to be converted into integers. 
target_train = target_train.astype('int')
target_test= target_test.astype('int')

target_train_HTC = target_train_HTC.astype('int')
target_test_HTC = target_test_HTC.astype('int')

target_train_Fe = target_train_Fe.astype('int')
target_test_Fe = target_test_Fe.astype('int')

target_train_ConvSC = target_train_ConvSC.astype('int')
target_test_ConvSC = target_test_ConvSC.astype('int')


#Setting up the random forest classifier.
Classifier = RandomForestClassifier(n_estimators = 100, max_depth = 100, random_state = 0)
#Classifier = RandomForestClassifier(n_estimators = 1000, random_state = 0)
all_data_class = Classifier.fit(data_train, (target_train))

Classifier_HTC = RandomForestClassifier(n_estimators = 100, max_depth = 100, random_state = 0)
#Classifier_HTC = RandomForestClassifier(n_estimators = 1000, random_state = 0)
HTC_class = Classifier_HTC.fit(data_train_HTC, (target_train_HTC))

Classifier_Fe = RandomForestClassifier(n_estimators = 100, max_depth = 100, random_state = 0)
#Classifier_Fe = RandomForestClassifier(n_estimators = 1000, random_state = 0)
Fe_class = Classifier_Fe.fit(data_train_Fe, (target_train_Fe))

Classifier_ConvSC = RandomForestClassifier(n_estimators = 100, max_depth = 100, random_state = 0)
#Classifier_ConvSC = RandomForestClassifier(n_estimators = 1000, random_state = 0)
ConvSC_class = Classifier_ConvSC.fit(data_train_ConvSC, (target_train_ConvSC))


Orginally, a max depth of 2 was used for for the random forest classifier, but the predictions were not very accurate. It has been run using a max depth of 2, 5, 10, 50, 100, and then no limit. N_estimators was increased from 100 to 1000. There were few noticable changes once max depth was greater than 50 and n_estimators greater than 100. If the classifier lines above are uncommented, they take significantly more time to run that the current script, with generally the same cross validation scores. 

In [ ]:
#Predicting the critical temps
target_pred = all_data_class.predict(data_test)
target_pred_HTC = HTC_class.predict(data_test_HTC)
target_pred_Fe = Fe_class.predict(data_test_Fe)
target_pred_ConvSC = ConvSC_class.predict(data_test_ConvSC)

#Setting up lists to be appended, in case multiple runs need to be done and averaged
score_all = []
score_HTC = []
score_Fe = []
score_ConvSC = []

cv_score_all = []
cv_score_HTC = []
cv_score_Fe = []
cv_score_ConvSC = []

#Caluculating the accuracy of the prediction
clf = accuracy_score(target_test, target_pred)
clf_HTC = accuracy_score(target_test_HTC, target_pred_HTC)
clf_Fe = accuracy_score(target_test_Fe, target_pred_Fe)
clf_ConvSC = accuracy_score(target_test_ConvSC, target_pred_ConvSC)

score_all.append(clf)
score_HTC.append(clf_HTC)
score_Fe.append(clf_Fe)
score_ConvSC.append(clf_ConvSC)

#Verifying the RandomTreeClassifier with cross validation. 
test_cv_score = cross_val_score(all_data_class, data_test, target_test)
test_cv_score_HTC = cross_val_score(HTC_class, data_test_HTC, target_test_HTC)
test_cv_score_Fe = cross_val_score(Fe_class, data_test_Fe, target_test_Fe)
test_cv_score_ConvSC = cross_val_score(ConvSC_class, data_test_ConvSC, target_test_ConvSC)

cv_score_all.append(test_cv_score)
cv_score_HTC.append(test_cv_score_HTC)
cv_score_Fe.append(test_cv_score_Fe)
cv_score_ConvSC.append(test_cv_score_ConvSC)

In [ ]:

#Displaying importance
feature_imp = pd.Series(all_data_class.feature_importances_, index = list(all_data_props)).sort_values(ascending=False)
feature_imp_HTC = pd.Series(HTC_class.feature_importances_, index = list(HTC_props)).sort_values(ascending=False)
feature_imp_Fe = pd.Series(Fe_class.feature_importances_, index = list(Fe_props)).sort_values(ascending=False)
feature_imp_ConvSC = pd.Series(ConvSC_class.feature_importances_, index = list(ConvSC_props)).sort_values(ascending=False)
#feature_imp

#Displaying the 20 most important features
feature_imp_20 = feature_imp[0:20]
feature_imp_20_HTC = feature_imp_HTC[0:20]
feature_imp_20_Fe = feature_imp_Fe[0:20]
feature_imp_20_ConvSC = feature_imp_ConvSC[0:20]

sns.barplot(x=feature_imp_20, y=feature_imp.index[0:20])
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features - All Data")
#plt.legend()
plt.show()

sns.barplot(x=feature_imp_20_HTC, y=feature_imp_HTC.index[0:20])
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features - HTC")
#plt.legend()
plt.show()

sns.barplot(x=feature_imp_20_Fe, y=feature_imp_Fe.index[0:20])
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features - Iron SCs")
#plt.legend()
plt.show()

sns.barplot(x=feature_imp_20_ConvSC, y=feature_imp_ConvSC.index[0:20])
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features - Conventional SCs")
#plt.legend()
plt.show()

print('All Data')
print(score_all)
print(cv_score_all)
print()
print('HTC')
print(score_HTC)
print(cv_score_HTC)
print()
print('Iron SCs')
print(score_Fe)
print(cv_score_Fe)
print()
print('Conventional SCs')
print(score_ConvSC)
print(cv_score_ConvSC)
print()

The important features generally agree with the Table 5 in the Hamidieh paper. 

In [ ]:
#Plotting the Random forest data
plt.scatter(target_test,target_pred)
plt.xlabel('Observed Critical Temp')
plt.ylabel('Predicted Critical Temp')
plt.title("Showing Observed vs Predicted Critical Temps for All Data")
ylim = (0,150)
xlim = (0,150)
plt.show()

plt.scatter(target_test_HTC,target_pred_HTC)
plt.xlabel('Observed Critical Temp')
plt.ylabel('Predicted Critical Temp')
plt.title("Showing Observed vs Predicted Critical Temps for HTC")
ylim = (0,150)
xlim = (0,150)
plt.show()

plt.scatter(target_test_Fe,target_pred_Fe)
plt.xlabel('Observed Critical Temp')
plt.ylabel('Predicted Critical Temp')
plt.title("Showing Observed vs Predicted Critical Temps for Iron SCs")
ylim = (0,150)
xlim = (0,150)
plt.show()

plt.scatter(target_test_ConvSC,target_pred_ConvSC)
plt.xlabel('Observed Critical Temp')
plt.ylabel('Predicted Critical Temp')
plt.title("Showing Observed vs Predicted Critical Temps for Conventional SCs")
ylim = (0,150)
xlim = (0,150)
plt.show()

Important Note: The plot for the conventional superconductors looks so strange because the floats had to be converted into whole number integers to be handled by the random forest classifier. The spread is not unreasonable for the number of elements contained in this data set, but no density can be determined for each point (there is no depth to each point indicating how many points actually lay at each coordinate).  

For the prediction of Table 6: I do not know how to insert/find the information needed for those materials. The code I would use to analyse it would be:

"
CsBe... = (properties of CsBe)

target_pred_CsBe = all_data_class.predict(CsBe)
"

and the resulting value would be the predicted temperature. I simply do not know how to find the information, or force the material information into python. 


Part 5 - Other machine learning models


In [ ]:
from sklearn.linear_model import Ridge

#Running through all categories with the Ridge package to compare to the random forest. 
reg = Ridge(alpha = 1000).fit(data_train, target_train)
reg_test_cr_scorei = cross_val_score(reg, data_test, target_test)
reg_test_cr_score = np.mean(reg_test_cr_scorei)

target_pred_lin = reg.predict(data_test)

reg = Ridge(alpha = 1000).fit(data_train_HTC, target_train_HTC)
reg_test_cr_scorei = cross_val_score(reg, data_test_HTC, target_test_HTC)
reg_test_cr_score_HTC = np.mean(reg_test_cr_scorei)

target_pred_lin_HTC = reg.predict(data_test_HTC)

reg = Ridge(alpha = 1000).fit(data_train_Fe, target_train_Fe)
reg_test_cr_scorei = cross_val_score(reg, data_test_Fe, target_test_Fe)
reg_test_cr_score_Fe = np.mean(reg_test_cr_scorei)

target_pred_lin_Fe = reg.predict(data_test_Fe)

reg = Ridge(alpha = 100).fit(data_train_ConvSC, target_train_ConvSC)
reg_test_cr_scorei = cross_val_score(reg, data_test_ConvSC, target_test_ConvSC)
reg_test_cr_score_ConvSC = np.mean(reg_test_cr_scorei)

target_pred_lin_ConvSC = reg.predict(data_test_ConvSC)

print()
print('The average cross_val score comparing the whole data set using Ridge is: ', reg_test_cr_score)
print()
print('The average cross_val score comparing the whole data set using Random Forests is:', np.mean(cv_score_all))
print()
print('The average cross_val score comparing the HTC data set using Ridge is:', reg_test_cr_score_HTC)
print()
print('The average cross_val score comparing the HTC data set using Random Forests is:', np.mean(cv_score_HTC))
print()
print('The average cross_val score comparing the Iron SC data set using Ridge is:', reg_test_cr_score_Fe)
print()
print('The average cross_val score comparing the Iron SC data set using Random Forests is:', np.mean(cv_score_Fe))
print()
print('The average cross_val score comparing the Conventional SC data set using Ridge is:', reg_test_cr_score_ConvSC)
print()
print('The average cross_val score comparing the Conventional SC set using Random Forests is:', np.mean(cv_score_ConvSC))
print()

#PLotting
plt.ylim(-30,170)
plt.xlim(-10,170)
plt.scatter(target_test,target_pred_lin)
plt.xlabel('Observed Critical Temp')
plt.ylabel('Predicted Critical Temp')
plt.title("Showing Observed vs Predicted Critical Temps with Ridge")
plt.show()

plt.ylim(-30,170)
plt.xlim(-10,170)
plt.scatter(target_test_HTC,target_pred_lin_HTC)
plt.xlabel('Observed Critical Temp')
plt.ylabel('Predicted Critical Temp')
plt.title("Showing Observed vs Predicted Critical Temps for HTC with Ridge")
plt.show()

plt.ylim(-30,170)
plt.xlim(-10,170)
plt.scatter(target_test_Fe,target_pred_lin_Fe)
plt.xlabel('Observed Critical Temp')
plt.ylabel('Predicted Critical Temp')
plt.title("Showing Observed vs Predicted Critical Temps for Iron SCs with Ridge")
plt.show()

plt.ylim(-1,11)
plt.xlim(-1,11)
plt.scatter(target_test_ConvSC,target_pred_lin_ConvSC)
plt.xlabel('Observed Critical Temp')
plt.ylabel('Predicted Critical Temp')
plt.title("Showing Observed vs Predicted Critical Temps for Conventional SC with Ridge")
plt.show()

Ridge was used as the alternative machine learning technique. It generally yielded better results that the random forest method, at least in terms of the cross-validated score. The difference may be because there were not enough n_estimators, but more estimators than what were used took significant amounts of time. 